In [1]:
import pandas as pd #To create and manipulate data frames
import numpy as np

In [2]:
df=pd.read_csv('input_data.csv') #Taking the input csv file with the urls as a dataframe
df.head() #checking for correct import

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...


In [3]:
import requests #module used to simulate web requests such as gets
from bs4 import BeautifulSoup as bs #Used for webscraping

In [4]:
for i in df.index: #loops through each row and denotes current row index by i
    name=str(df['URL_ID'][i])#Stores URL_ID as name for the file being created
    url=str(df['URL'][i])#Stores URL in url
    page=requests.get(url)#requests the source file using the url
    soup=bs(page.content,'html.parser')#Takes the html source code and specifies html parser for parsing through it.
    try: #Error handling to try scraping the website, does not perform scraping if source file not found for website.
        try:
            title=soup.find("h1",class_='entry-title').text
            data=soup.find('div',class_='td-post-content tagdiv-type')
        except AttributeError:
            title=soup.find("h1",class_='tdb-title-text').text #finds alternative class name in case prev name not found.
            data=soup.find("div",class_='tdb-block-inner td-fix-index')
        
        data=data.find_all(['p','h1','h2','h3','h4','h5','li'])    
        title=title+'\n\n'
        file=open('/Users/omarhassan/Desktop/text_analysis/extracted/'+name,'w')#creates file with URL_ID as filename
        file.write(title)#writes the title into file
        for i in data:#loop that writes each paragraph into the file
            file.write(i.text)
            file.write('\n')
        file.close()#to close the file in order to preserve memory
    except:
        continue #Skips current URL or row due to unknown reason or no source file provided.

In [5]:
import nltk #Natural Language ToolKit used for prpcessing text data
import re #Regex for string manipulation

In [6]:
def avg_sentence_length(text,words): #returns the average sentence length based on the given formula
    sentences=re.split(r'[.\n]',text)
    length=[]
    for sentence in sentences:
        sentence=[word for word in sentence if word in words]
        length.append(len(sentence))
    return (sum(length)/len(sentences))

In [7]:
def count_syllables(word): #Returns the count of the syllables in the given word
    word = word.lower()
    counter = 0
    is_previous_vowel = False
    for index, value in enumerate(word):
        if value in ["a", "e", "i", "o", "u", "y"]:
            if index == len(word) - 1:
                if value == "e":
                    if counter == 0:
                        counter += 1
                else:
                    counter += 1
            else:
                if is_previous_vowel == True:
                    counter += 1
                    is_previous_vowel = False
                    break
            is_previous_vowel = True
        else:
            if is_previous_vowel == True:
                counter += 1
            is_previous_vowel = False
    return counter

def complex(word): #returns True if the word passed is a complex word
    if(count_syllables(word)>2):
        return True
    return False

In [8]:
def avg_words_per(text,words):#Returns the average number of words per sentence based on the given formulas
    sentences=re.split(r'[.\n]',text)
    return (len(words)/len(sentences))
    

In [32]:
def perc_complex(words):#A powerful function which returns a dictionary of vales related to the complex count, word count, polarity, subjectivity, positive & negative score, average word length and complex percentage.
    comp=[]
    avg=[]
    pos_score=0
    neg_score=0
    polarity=0
    subjectivity=0
    for word in words:
        if(complex(word)==True):
            comp.append(word)
        avg.append(len(word))
        if word in pos:
            pos_score+=1
        elif word in neg:
            neg_score-=1
    neg_score=-1*neg_score
    try:
        polarity=((pos_score-neg_score)/(pos_score+neg_score))+0.000001
    except:
        polarity=0.000001
    subjectivity=((pos_score+neg_score)/len(words))
    return {'comp_perc':(len(comp)/len(words))*100,'comp_count':len(comp),'word_count':len(words),'avg_wrd_len':(sum(avg)/len(avg)),'polarity':polarity,'subjectivity':subjectivity,'pos_score':pos_score,'neg_score':neg_score}

In [10]:
def syl_per_word(words): # Returns the syllables per word count
    syl=[]
    for word in words:
        count=count_syllables(word)
        syl.append(count)
    return sum(syl)

In [11]:
pronounRegex = re.compile(r'\bI\b|\bwe\b|\bWe\b|\bmy\b|\bMy\b|\bours\b|\bus\b') #used to find the pronouns quickly in a given text

The code below imports the positive word list and negative word list in order to calculate positive & negative score.

In [12]:
f_pos=open('/Users/omarhassan/Desktop/text_analysis/MasterDictionary/positive-words.txt','r',encoding = "ISO-8859-1")
f_neg=open('/Users/omarhassan/Desktop/text_analysis/MasterDictionary/negative-words.txt','r',encoding = "ISO-8859-1")

In [13]:
pos=f_pos.read()#stores the list as a string
neg=f_neg.read()
f_pos.close()#closes file to preserve memory
f_neg.close()

In [14]:
pos=pos.split('\n')#converts string to a list for easier comparison
pos

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

In [15]:
neg=neg.split('\n')#converts string to a list for easier comparison
neg

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted',
 'aborts',
 'abrade',
 'abrasive',
 'abrupt',
 'abruptly',
 'abscond',
 'absence',
 'absent-minded',
 'absentee',
 'absurd',
 'absurdity',
 'absurdly',
 'absurdness',
 'abuse',
 'abused',
 'abuses',
 'abusive',
 'abysmal',
 'abysmally',
 'abyss',
 'accidental',
 'accost',
 'accursed',
 'accusation',
 'accusations',
 'accuse',
 'accuses',
 'accusing',
 'accusingly',
 'acerbate',
 'acerbic',
 'acerbically',
 'ache',
 'ached',
 'aches',
 'achey',
 'aching',
 'acrid',
 'acridly',
 'acridness',
 'acrimonious',
 'acrimoniously',
 'acrimony',
 'adamant',
 'adamantly',
 'addict',
 'addicted',
 'addicting',
 'addicts',
 'admonish',
 'admonisher',
 'admonishingly',
 'admonishment',
 'admonition',
 'adulterate',
 'adulterated',
 'adulteration',
 'adulterier',
 'adversarial',
 'adversary',
 'adverse',
 'adversity',
 'afflict',
 'affliction',
 'afflictive',
 'affront',


In [16]:
f_stop=open('/Users/omarhassan/Desktop/text_analysis/StopWords/stopwords.txt','r')#The stopwords files were all combined to a single file called stopwords.
stopwords=f_stop.read()#converts text to string
f_stop.close()#preserves memory

In [17]:
stopwords=stopwords.split('\n')#converts string to a easily comparable list

In [18]:
def to_words(text):#returns a cleaner text formatted as a list which is free of stopwords.
    words=nltk.tokenize.word_tokenize(text)
    words=[word for word in words if word not in stopwords]
    return words

In [19]:
output=pd.read_csv('output.csv')#copies the outfput format and stores as a data frame
output.head()#check for correct import

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,38,https://insights.blackcoffer.com/what-if-the-c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,39,https://insights.blackcoffer.com/what-jobs-wil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40,https://insights.blackcoffer.com/will-machine-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41,https://insights.blackcoffer.com/will-ai-repla...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
output.columns #checks for correct number of columns

Index(['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
       'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH',
       'PERCENTAGE OF COMPLEX WORDS', 'FOG INDEX',
       'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT', 'WORD COUNT',
       'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'],
      dtype='object')

In [21]:
pd.options.mode.chained_assignment=None #removes warning for chained assignments

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/omarhassan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
for i in output.index: #modifies dataframe row by row.
    try:
        file=str(int(output['URL_ID'][i]))#stores URL_ID in order to access extracted files with the same name.
        f=open('/Users/omarhassan/Desktop/text_analysis/extracted/'+file,"r")#opens current rows webscraped file
        text=f.read() #converts file content to string
        words=to_words(text) #Converts to cleaned words
        result=perc_complex(words) #Runs the powerful function which returns a dictionary as output
        asl=avg_sentence_length(text,words) #calculates Average Sentence Length
        output['POSITIVE SCORE'][i]=result['pos_score'] #Stores positive score accordingly
        output['NEGATIVE SCORE'][i]=-1*result['neg_score'] #Stores Negative score accordingly
        output['POLARITY SCORE'][i]=result['polarity'] #Stores Polarity score accordingly
        output['SUBJECTIVITY SCORE'][i]=result['subjectivity'] #Stores subjectivity score accordingly
        output['AVG SENTENCE LENGTH'][i]= asl #Stores Average Sentence length accordingly
        output['PERCENTAGE OF COMPLEX WORDS'][i]=result['comp_perc'] #Stores Complex Word Percentage accordingly
        output['FOG INDEX'][i]=(asl+result['comp_perc'])*0.4 #Calculates and stores FOG Index
        output['AVG NUMBER OF WORDS PER SENTENCE'][i]=avg_words_per(text,words)#Calculates Average Number of Words per Sentence
        output['COMPLEX WORD COUNT'][i]=result['comp_count'] #Calculates Complex word count
        output['WORD COUNT'][i]=result['word_count'] #Stores cleaned word count accordingly
        output['SYLLABLE PER WORD'][i]=syl_per_word(words) #stores Syllable per word
        output['PERSONAL PRONOUNS'][i]=len(pronounRegex.findall(text)) #Stores count for Personal Pronouns
        output['AVG WORD LENGTH'][i]=result['avg_wrd_len'] #Stores Average word length
        f.close() #Preserves memory
    except:
        continue

In [38]:
output.head() #checks output

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,37,https://insights.blackcoffer.com/ai-in-healthc...,63.0,-33.0,0.312501,0.076800,1.216216,31.280000,12.998486,11.261261,391.0,1250.0,2307.0,1.0,6.384800
1,38,https://insights.blackcoffer.com/what-if-the-c...,56.0,-38.0,0.191490,0.109302,2.102273,22.558140,9.864165,9.772727,194.0,860.0,1306.0,6.0,5.308140
2,39,https://insights.blackcoffer.com/what-jobs-wil...,65.0,-35.0,0.300001,0.088810,1.280702,30.994671,12.910149,9.877193,349.0,1126.0,2081.0,3.0,6.158970
3,40,https://insights.blackcoffer.com/will-machine-...,57.0,-25.0,0.390245,0.088172,0.831933,22.258065,9.235999,7.815126,207.0,930.0,1484.0,17.0,5.580645
4,41,https://insights.blackcoffer.com/will-ai-repla...,49.0,-23.0,0.361112,0.065753,1.713115,25.205479,10.767438,8.975410,276.0,1095.0,1806.0,16.0,5.720548


In [39]:
output.to_excel('final_output.xlsx') #converts output to excel file name 'final_output'